In [4]:
import numpy as np
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, average_precision_score, confusion_matrix
import sklearn.metrics as metrics
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression

In [5]:
metrics_to_calculate = ['auroc', 'aupr']


In [6]:
drug_features = np.loadtxt('/Users/mac/OneDrive/thesis/scripts/dataset/DPI_Gprotein/dpig_X1.txt')
drug_features

array([[1.        , 0.2826283 , 0.08349522, ..., 0.01975119, 0.06815311,
        0.19913334],
       [0.2826283 , 1.        , 0.06314988, ..., 0.02294885, 0.05182183,
        0.2035093 ],
       [0.08349522, 0.06314988, 1.        , ..., 0.01932024, 0.05582016,
        0.04895691],
       ...,
       [0.01975119, 0.02294885, 0.01932024, ..., 1.        , 0.02354822,
        0.0296562 ],
       [0.06815311, 0.05182183, 0.05582016, ..., 0.02354822, 1.        ,
        0.05182548],
       [0.19913334, 0.2035093 , 0.04895691, ..., 0.0296562 , 0.05182548,
        1.        ]])

In [7]:
interaction_matrix = np.loadtxt('/Users/mac/OneDrive/thesis/scripts/dataset/DPI_Gprotein/dpig_Y.txt')
interaction_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [8]:
metric_values_per_fold = {}
if 'auroc' in metrics_to_calculate:
    metric_values_per_fold['auroc_micro'] = []
    metric_values_per_fold['auroc_macro'] = []
if 'aupr' in metrics_to_calculate:
    metric_values_per_fold['aupr_micro'] = []
    metric_values_per_fold['aupr_macro'] = []

In [9]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)

In [10]:
fold_counter = 0
for train_index, test_index in kf.split(drug_features):
    print('======================= Fold '+str(fold_counter)+' =======================')
    
    # split the dataset
    X_train, X_test = drug_features[train_index], drug_features[test_index]
    y_train, y_test = interaction_matrix[train_index], interaction_matrix[test_index]
    
    # define the oneVSrest classifier with the base classifier
    clf = OneVsRestClassifier(RandomForestClassifier())
    
    #clf = OneVsRestClassifier(LogisticRegression(random_state=0))
    
    # fit the classifier on the training set
    clf.fit(X_train, y_train)
    
    # generate probability predictions for every sample in the test set
    y_pred = clf.predict_proba(X_test)
    
    print(str(y_pred.shape))
    
    # calculate the performance metrics on the test set
    if 'auroc' in metrics_to_calculate:
        metric_values_per_fold['auroc_micro'].append(roc_auc_score(y_test, y_pred, average='micro'))
        
        # This is not really important as we are only interested in the micro measures.
        # Nevertheless, I basically do the macro averaging by hand so that I can skip labels that have only samples with one class
        roc_auc_per_label = []
        for label_idx in range(interaction_matrix.shape[1]):
            if len(set(y_test[:, label_idx])) >= 2:
                roc_auc_per_label.append(roc_auc_score(y_test[:, label_idx], y_pred[:, label_idx]))
        print(str(len(roc_auc_per_label))+' out of the '+str(y_test.shape[1])+' total labels has more than one classes present')
        
        metric_values_per_fold['auroc_macro'].append(np.mean(roc_auc_per_label))

        
    if 'aupr' in metrics_to_calculate:
        metric_values_per_fold['aupr_micro'].append(average_precision_score(y_test, y_pred, average='micro'))
        
        aupr_per_label = []
        for label_idx in range(interaction_matrix.shape[1]):
            if len(set(y_test[:, label_idx])) >= 2:
                aupr_per_label.append(average_precision_score(y_test[:, label_idx], y_pred[:, label_idx]))
        
        metric_values_per_fold['aupr_macro'].append(np.mean(aupr_per_label))

    
    fold_counter += 1
    print('========================================================================')
    print('')

======================= Fold 0 =======================
(10, 223)
60 out of the 223 total labels has more than one classes present

======================= Fold 1 =======================
(10, 223)
67 out of the 223 total labels has more than one classes present

======================= Fold 2 =======================
(10, 223)
62 out of the 223 total labels has more than one classes present

======================= Fold 3 =======================
(10, 223)
42 out of the 223 total labels has more than one classes present

======================= Fold 4 =======================
(10, 223)
86 out of the 223 total labels has more than one classes present

======================= Fold 5 =======================
(9, 223)
22 out of the 223 total labels has more than one classes present

======================= Fold 6 =======================
(9, 223)
24 out of the 223 total labels has more than one classes present

======================= Fold 7 =======================
(9, 223)
42 out of the 223 tot

In [11]:
metric_values_per_fold

{'auroc_micro': [0.8005691432087372,
  0.9333683769080078,
  0.7761687776748019,
  0.909435261707989,
  0.8523912829444338,
  0.9603714910068919,
  0.8063340389374198,
  0.6383792048929664,
  0.821747465728242,
  0.906439586903163],
 'auroc_macro': [0.8135416666666667,
  0.9229003790570955,
  0.8155801971326164,
  0.9246031746031745,
  0.8266493632336654,
  0.9545454545454546,
  0.78125,
  0.684311224489796,
  0.8246173469387755,
  0.9127906976744186],
 'aupr_micro': [0.2791824455888884,
  0.7181381742244055,
  0.5318738886999742,
  0.5905404975998026,
  0.5734453546691645,
  0.702899977851615,
  0.5179722016336363,
  0.3015611069293883,
  0.2596979142727353,
  0.4133263365340298],
 'aupr_macro': [0.6188888888888889,
  0.8445273631840796,
  0.6786482334869431,
  0.8317460317460317,
  0.6852390180878554,
  0.8686868686868686,
  0.638888888888889,
  0.4457671957671958,
  0.6259424603174603,
  0.7642118863049097]}

In [12]:
# calculate the mean and std for every metric measured during training and validation
for metric_name in metric_values_per_fold.keys():
    print(metric_name+': '+ str(np.mean(metric_values_per_fold[metric_name])) +' ('+ str(np.std(metric_values_per_fold[metric_name])) +')')
    print('')

auroc_micro: 0.8405204629912653 (0.0896325685104234)

auroc_macro: 0.8460789504341664 (0.07841018648994735)

aupr_micro: 0.48886378980036393 (0.15994969893477)

aupr_macro: 0.7002546835359122 (0.1232595455378644)

